In [3]:
import numpy as np
import random

In [4]:
def kMedoids(D, k, tmax=100):
    m, n = D.shape

    if k > n:
        raise Exception('too many medoids')

    valid_medoid_inds = set(range(n))
    invalid_medoid_inds = set([])
    rs,cs = np.where(D==0)

    index_shuf = list(range(len(rs)))
    np.random.shuffle(index_shuf)
    rs = rs[index_shuf]
    cs = cs[index_shuf]
    for r,c in zip(rs,cs):

        if r < c and r not in invalid_medoid_inds:
            invalid_medoid_inds.add(c)
    valid_medoid_inds = list(valid_medoid_inds - invalid_medoid_inds)

    if k > len(valid_medoid_inds):
        raise Exception('too many medoids (after removing {} duplicate points)'.format(
            len(invalid_medoid_inds)))

    M = np.array(valid_medoid_inds)
    np.random.shuffle(M)
    M = np.sort(M[:k])

    Mnew = np.copy(M)

    C = {}
    for t in range(tmax):

        J = np.argmin(D[:,M], axis=1)
        for kp in range(k):
            C[kp] = np.where(J==kp)[0]

        for kp in range(k):
            J = np.mean(D[np.ix_(C[kp],C[kp])],axis=1)
            j = np.argmin(J)
            Mnew[kp] = C[kp][j]
        np.sort(Mnew)

        if np.array_equal(M, Mnew):
            break
        M = np.copy(Mnew)
    else:

        J = np.argmin(D[:,M], axis=1)
        for kp in range(k):
            C[kp] = np.where(J==kp)[0]

    return M, C

In [5]:
data = np.array([[1,1], 
                [2,2], 
                [10,10]])
from sklearn.metrics.pairwise import pairwise_distances

D = pairwise_distances(data, metric='euclidean')
print(D)

M, C = kMedoids(D, 2)

print('medoids:')
for point_idx in M:
    print( data[point_idx] )

print('')
print('clustering result:')
for label in C:
    for point_idx in C[label]:
        print('label {0}:　{1}'.format(label, data[point_idx]))



[[ 0.          1.41421356 12.72792206]
 [ 1.41421356  0.         11.3137085 ]
 [12.72792206 11.3137085   0.        ]]
medoids:
[1 1]
[2 2]

clustering result:
label 0:　[1 1]
label 1:　[2 2]
label 1:　[10 10]
